# Day 16
https://adventofcode.com/2020/day/16

In [1]:
import aocd
data = aocd.get_data(year=2020, day=16)

In [2]:
from dataclasses import dataclass
from math import prod
import re
from typing import Tuple

In [3]:
@dataclass(frozen=True)
class Rule():
    field: str
    ranges: Tuple[Tuple[int]]
    
    @classmethod
    def from_input(cls, field, ranges):
        ranges = tuple(tuple(map(int, rg)) for rg in ranges)
        return cls(field, ranges)
    
    def valid_value(self, value):
        return any(lower <= value <= upper for lower, upper in self.ranges)

In [4]:
re_rule = re.compile(r'(.+): (.+)')
re_ranges = re.compile(r'(\d+)-(\d+)')
def read_rules(text):
    return tuple(Rule.from_input(field, re_ranges.findall(ranges)) for field, ranges in re_rule.findall(text))

In [5]:
re_ticket = re.compile(r'(\d+)')
def read_my_ticket(text):
    return tuple(map(int, re_ticket.findall(text)))

In [6]:
def read_nearby_tickets(text):
    return tuple(tuple(map(int, line.split(','))) for line in text.split('\n')[1:])

In [7]:
def filter_tickets(tickets, rules):
    invalid_tickets = set()
    error_rate = 0
    
    for ix, ticket in enumerate(tickets):
        invalid_values = sum(value for value in ticket if not any(rule.valid_value(value) for rule in rules))
        if invalid_values > 0:
            invalid_tickets.add(ix)
            error_rate += invalid_values
    
    return error_rate, tuple(ticket for ix, ticket in enumerate(tickets) if ix not in invalid_tickets)

In [8]:
def possible_fields(tickets, column, rules):
    values = [ticket[column] for ticket in tickets]
    return tuple(rule.field for rule in rules
                 if all(rule.valid_value(value) for value in values))

In [9]:
def identify_fields(tickets, rules):
    columns = [possible_fields(tickets, column, rules) for column in range(len(tickets[0]))]
    while max(len(possible) for possible in columns) > 1:
        for ix, possible in enumerate(columns):
            if len(possible) == 1:
                columns = [tuple(f for f in fields if f != possible[0]) if len(fields) > 1 else fields
                           for fields in columns]
    return tuple(column[0] for column in columns)

In [10]:
def departure_product(ticket, columns):
    return prod(value for value, column in zip(ticket, columns) if column[:9] == 'departure')

In [11]:
sections = data.split('\n\n')
rules = read_rules(sections[0])
my_ticket = read_my_ticket(sections[1])
nearby_tickets = read_nearby_tickets(sections[2])

p1, tickets = filter_tickets(nearby_tickets, rules)
print('Part 1: {}'.format(p1))

fields = identify_fields(tickets, rules)
p2 = departure_product(my_ticket, fields)
print('Part 2: {}'.format(p2))

Part 1: 32842
Part 2: 2628667251989
